In [1]:
import sys
import os

# Add parent directory
parent_dir = os.path.dirname(os.getcwd())
if parent_dir not in sys.path:
    sys.path.insert(0, parent_dir)

os.chdir(parent_dir)

from models.base import SessionLocal
from models import UseCase, Company, Industry, Person
from typing import Optional, List, Dict, Any

# Use Case Service Class
Should implement all interaction functionality with db

In [2]:
class UseCaseService:
    """
    Layer that is intented to handle all interaction with the database for managin usecases. 
    CRUD operation.
    """
    def __init__(self):
        self.valid_status_values = [
            "new",
            "in_review",
            "approved",
            "in_progress",
            "completed",
            "archived"
        ]

    def _get_session(self):
        """Helper to get database session"""
        return SessionLocal()  # may get more complicated dont know, if so can be handeled centrally here
    
    def _validate_status(self, status : str) -> None:
        if status not in self.valid_status_values:
            valid_status_valus = ", ".join(self.valid_status_values)
            raise ValueError(f"Given status '{status}' is not valid. Please choose one of the following status values '{valid_status_valus}'")
    
    def _use_case_to_dict(self, use_case : UseCase) -> Dict[str, Any]: 
        """
        Helper function translating a use case to a dict. 
        Args: 
            use_case : Use case abj

        Returns: 
            Dict containing use case information EXCEPT persons involved
        """
        return {
            "id": use_case.id,
            "title": use_case.title,
            "description": use_case.description,
            "expected_benefit": use_case.expected_benefit,
            "status": use_case.status,
            "company_id": use_case.company_id,
            "company_name": use_case.company.name,
            "industry_id": use_case.industry_id,
            "industry_name": use_case.industry.name
        }


    
    def get_all_use_cases(self) -> List[Dict[str, Any]]: 
        # get db
        db = self._get_session()

        # try to get all use cases and format them reasonably
        try: 
            use_cases = db.query(UseCase).all()
            print(use_cases)
            return [self._use_case_to_dict(uc) for uc in use_cases]
        finally:
            db.close()

    def get_use_case_by_id(self, use_case_id : int) -> Optional[Dict[str, Any]]:
        """
        get the use case for the identifier provided.
        """
        
        db = self._get_session()

        try: 
            matching_use_case = db.query(UseCase).filter(UseCase.id == use_case_id).first()

            if not matching_use_case:
                return None
            else:
                return self._use_case_to_dict(matching_use_case)
        finally:
            db.close()

    def create_use_case(self, title : str, company_id : int, industry_id : int, description : str = None, expected_benefit : str = None, status : str  = 'new') -> Dict[str, Any]:

        db = self._get_session()

        try:

            # checks
            # (1) is company existing?
            company = db.query(Company).filter(Company.id == company_id).first()
            if not company:
                raise ValueError(f"Company with ID {company_id} does not exist.")
            
            # (2) is industry existing?
            industry = db.query(Industry).filter(Industry.id == industry_id).first()
            if not industry:
                raise ValueError(f"Industry with ID {industry_id} does not exist.")
            
            # (3) title may not be empty
            if len(title) == 0:
                raise ValueError("Title must not be empty.")

            # (4) status in valid range
            self._validate_status(status)

            # if ok lets create a new use case
            new_use_case = UseCase(
                title = title,
                description = description, 
                expected_benefit = expected_benefit, 
                company_id = company_id, 
                industry_id = industry_id, 
                status = status
            )
            
            # add and save
            db.add(new_use_case)
            db.commit()
            db.refresh(new_use_case)

            return self._use_case_to_dict(new_use_case)
        
        except Exception as e:  # hope thats alright TODO check if rollback is correct or if there is no error handling needed
            db.rollback()
            raise e
        
        finally:
            db.close()
            
            
    def __repr__(self):
        return "<UseCaseService>"

In [3]:
service = UseCaseService()

# test get all use cases

In [4]:
print(service.get_all_use_cases())

2026-02-13 15:52:55,822 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2026-02-13 15:52:55,824 INFO sqlalchemy.engine.Engine SELECT use_cases.id AS use_cases_id, use_cases.title AS use_cases_title, use_cases.description AS use_cases_description, use_cases.expected_benefit AS use_cases_expected_benefit, use_cases.status AS use_cases_status, use_cases.company_id AS use_cases_company_id, use_cases.industry_id AS use_cases_industry_id 
FROM use_cases
2026-02-13 15:52:55,825 INFO sqlalchemy.engine.Engine [generated in 0.00064s] ()
[<UseCase(id=1, title='Patient protal', status='new', company_id=1)>, <UseCase(id=2, title='AI Diagnostics', status='in_progress', company_id=2)>, <UseCase(id=3, title='EHR Migration', status='completed', company_id=3)>, <UseCase(id=4, title='Cloud Migration', status='new', company_id=4)>, <UseCase(id=5, title='Mobile App Dev', status='in_progress', company_id=5)>, <UseCase(id=6, title='Cybersecurity Program', status='new', company_id=6)>, <UseCase(id=7, title='Io

# test get use case by id

In [5]:
use_case_2 = service.get_use_case_by_id(999)

2026-02-13 15:52:55,846 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2026-02-13 15:52:55,847 INFO sqlalchemy.engine.Engine SELECT use_cases.id AS use_cases_id, use_cases.title AS use_cases_title, use_cases.description AS use_cases_description, use_cases.expected_benefit AS use_cases_expected_benefit, use_cases.status AS use_cases_status, use_cases.company_id AS use_cases_company_id, use_cases.industry_id AS use_cases_industry_id 
FROM use_cases 
WHERE use_cases.id = ?
 LIMIT ? OFFSET ?
2026-02-13 15:52:55,848 INFO sqlalchemy.engine.Engine [generated in 0.00065s] (999, 1, 0)
2026-02-13 15:52:55,849 INFO sqlalchemy.engine.Engine ROLLBACK


# create new usecase

In [6]:
new_uc = service.create_use_case(
    title="Test Use Case - Automated Testing",
    description="Implement automated testing framework for quality assurance",
    expected_benefit="Reduce bugs by 50%, faster deployment cycles",
    company_id=6,
    industry_id=2,
    status="new"
)

2026-02-13 15:52:55,855 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2026-02-13 15:52:55,856 INFO sqlalchemy.engine.Engine SELECT companies.id AS companies_id, companies.name AS companies_name, companies.industry_id AS companies_industry_id 
FROM companies 
WHERE companies.id = ?
 LIMIT ? OFFSET ?
2026-02-13 15:52:55,857 INFO sqlalchemy.engine.Engine [generated in 0.00058s] (6, 1, 0)
2026-02-13 15:52:55,858 INFO sqlalchemy.engine.Engine SELECT industries.id AS industries_id, industries.name AS industries_name 
FROM industries 
WHERE industries.id = ?
 LIMIT ? OFFSET ?
2026-02-13 15:52:55,859 INFO sqlalchemy.engine.Engine [generated in 0.00050s] (2, 1, 0)
2026-02-13 15:52:55,860 INFO sqlalchemy.engine.Engine INSERT INTO use_cases (title, description, expected_benefit, status, company_id, industry_id) VALUES (?, ?, ?, ?, ?, ?)
2026-02-13 15:52:55,860 INFO sqlalchemy.engine.Engine [generated in 0.00035s] ('Test Use Case - Automated Testing', 'Implement automated testing framework for qu

In [11]:
retrieved = service.get_use_case_by_id(new_uc['id'])


service.create_use_case(
        title="Should Fail",
        description="This should fail",
        expected_benefit="None",
        company_id=20000,  # Invalid ID
        industry_id=1,
        status="new" # invalid status
    )

service.create_use_case(
        title="Should Fail",
        description="This should fail",
        expected_benefit="None",
        company_id=2,  # Invalid ID
        industry_id=1,
        status="newd" # invalid status
    )

2026-02-13 15:54:57,297 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2026-02-13 15:54:57,297 INFO sqlalchemy.engine.Engine SELECT use_cases.id AS use_cases_id, use_cases.title AS use_cases_title, use_cases.description AS use_cases_description, use_cases.expected_benefit AS use_cases_expected_benefit, use_cases.status AS use_cases_status, use_cases.company_id AS use_cases_company_id, use_cases.industry_id AS use_cases_industry_id 
FROM use_cases 
WHERE use_cases.id = ?
 LIMIT ? OFFSET ?
2026-02-13 15:54:57,298 INFO sqlalchemy.engine.Engine [cached since 121.5s ago] (10, 1, 0)
2026-02-13 15:54:57,299 INFO sqlalchemy.engine.Engine SELECT companies.id AS companies_id, companies.name AS companies_name, companies.industry_id AS companies_industry_id 
FROM companies 
WHERE companies.id = ?
2026-02-13 15:54:57,299 INFO sqlalchemy.engine.Engine [cached since 121.5s ago] (6,)
2026-02-13 15:54:57,300 INFO sqlalchemy.engine.Engine SELECT industries.id AS industries_id, industries.name AS industr

ValueError: Company with ID 20000 does not exist.